<a href="https://colab.research.google.com/github/Yhabib05/Chatbot-/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
from google.colab import drive
drive.mount('/content/drive')
the_data_file_path='/content/drive/My Drive/chatbot/intents.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import json
import string
import random

import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [72]:
#Loading the data
our_file=open(the_data_file_path).read()
data=json.loads(our_file)

In [73]:
type(data)

dict

In [74]:
print(len(data))

1


In [75]:
np.shape(data)

()

In [76]:
print(data)

{'intents': [{'tag': 'hello', 'patterns': ['Hello', 'Hi there', 'Good morning', 'Good evening', "What's up", 'Howdy', 'Greetings', 'Hey', 'Yo', 'Hi, how are you?'], 'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!', 'Greetings!', 'Hi there! How can I assist you?', "Hey! What's up?", "Yo, what's good?"], 'context': ''}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, I can't understand you.", 'Please give me more info.', "I'm not sure I understand.", 'Can you please clarify?', "I didn't quite catch that, can you repeat?"], 'context': ['']}, {'tag': 'job', 'patterns': ['What is your job?', 'What do you work as?', 'What is your work?', 'Tell me about your job', 'What do you do?', "What's your occupation?"], 'responses': ['My job is to make you feel like everything is okay.', 'I work to serve you as well as possible.', "I'm here to assist you with anything you need.", 'My role is to help you find information and perform tasks.'], 'context': ''}, {'tag': 'age', 'patterns': ['W

# Patterns = Questions

In [77]:
print(data['intents'][0])

{'tag': 'hello', 'patterns': ['Hello', 'Hi there', 'Good morning', 'Good evening', "What's up", 'Howdy', 'Greetings', 'Hey', 'Yo', 'Hi, how are you?'], 'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!', 'Greetings!', 'Hi there! How can I assist you?', "Hey! What's up?", "Yo, what's good?"], 'context': ''}


In [78]:
# creating our vocabulary lists
words=[] # all the possible words
tags=[] #tags to choose from

data_x=[] #input
data_y=[] # the output what we need to guess

for intent in data['intents']:
  for pattern in intent['patterns']:
    tokens=nltk.word_tokenize(pattern) #separating the pattern(phrase) into a list of words
    words.extend(tokens)
    data_x.append(pattern)
    data_y.append(intent['tag']),

  if intent['tag'] not in tags:
    tags.append(intent['tag'])

lemmatizer=WordNetLemmatizer()
words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words=sorted(set(words))
tags=sorted(set(tags))

print(data_x)
print(data_y)

['Hello', 'Hi there', 'Good morning', 'Good evening', "What's up", 'Howdy', 'Greetings', 'Hey', 'Yo', 'Hi, how are you?', 'What is your job?', 'What do you work as?', 'What is your work?', 'Tell me about your job', 'What do you do?', "What's your occupation?", 'What is your age?', 'How old are you?', 'When were you born?', 'Tell me your age', 'Are you old?', 'How are you today?', 'How are you?', 'How are you feeling?', "How's it going?", 'You good?', 'How do you feel?', 'I am good too', 'I feel fine', 'Good!', 'Fine', 'I am good', 'I am great', 'Feeling awesome', "I'm doing well", 'Great!', 'I am feeling bad', 'No, I am sad', 'Not good', "I'm feeling down", "I'm upset", "No, I'm not okay", 'What can you do?', 'What can I ask you?', 'Can you help me?', 'What are your skills?', 'What tasks can you perform?', 'How can you assist me?', 'Are you a girl?', 'Are you a woman?', 'Are you a boy?', 'Are you male or female?', 'What is your gender?', 'Thank you', 'Thank you very much', 'Thanks', 'I

# Bag of words

In [79]:
#transforming the text to numbers
training=[]
representation_tag=[0]*len(tags)
representation_word_def=[0]*len(words)
for idx, doc in enumerate(data_x):

  text=lemmatizer.lemmatize(doc.lower())
  representation_word=list(representation_word_def)

  for word in nltk.word_tokenize(text):
    if word in words:
      representation_word[words.index(word) ] = 1

  output_row=list(representation_tag)
  output_row[tags.index(data_y[idx])]=1
  training.append([representation_word, output_row])

random.shuffle(training)
training=np.array(training, dtype=object)

train_x=np.array(list(training[:,0]))
train_y=np.array(list(training[:,1]))



In [80]:
print(len(train_x_[0]))
print(len(words)) #they are the same length bcs
#an element of train_x is 0110..011: a representation of the word
# within the vocabulary(words)
print(train_y)

58
104
[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


# The neural Network

In [81]:
model=Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

adam=tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.fit(x=train_x, y=train_y, epochs=150, verbose=1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0561 - loss: 2.7165
Epoch 2/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1779 - loss: 2.5879 
Epoch 3/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2892 - loss: 2.4010 
Epoch 4/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2843 - loss: 2.2682 
Epoch 5/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4890 - loss: 2.0040 
Epoch 6/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5344 - loss: 1.7579 
Epoch 7/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6213 - loss: 1.4698 
Epoch 8/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5878 - loss: 1.3357 
Epoch 9/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5688 - loss: 1.2370 
Epoch 10/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6969 - loss: 1.0182 
Epoch 11/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6379 - loss: 0.9566 
Epoch 12/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6979 - loss

In [82]:
#input of the user
def clean_text(text):
  tokens=nltk.word_tokenize(text)
  tokens=[lemmatizer.lemmatize(word) for word in tokens]
  return tokens


def bag_of_words(text, vocab):
  tokens=clean_text(text)
  representation_word=[0]*len(vocab)
  for word in tokens:
    for idx, w in enumerate(vocab):
      if w==word:
        representation_word[idx]=1
  return np.array(representation_word)

def prediction(text, vocab, labels):
  bow=bag_of_words(text, vocab)
  result=model.predict(np.array([bow]))[0] # return a 2D array [[0.8, 0.7, 0.2]] the probabilities for all the tags
  thresh=0.2
  y_pred=[[idx, res] for idx, res in enumerate(result) if res>thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  prediction_labels=[]
  for r in y_pred:
    prediction_labels.append(labels[r[0]])
  return prediction_labels

def get_response(intents_list, intents_json):
  tag=intents_list[0]
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result

In [84]:
# executing the code
print("Press 0 to Quit")
while True:
  message=input("")
  if message=='0':
    break
  intents=prediction(message, words, tags)
  result=get_response(intents, data)
  print(result)

Press 0 to Quit
hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Hi there! How can I assist you?
what is your job ? 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
My role is to help you find information and perform tasks.
your job ? $
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
I'm here to assist you with anything you need.
really ? 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Good morning!
how are you ? 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Very good, and you?
good 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Good to know you're doing well!
0
